<a href="https://colab.research.google.com/github/Luke-687/Portfolio/blob/main/MowingTheLawnRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Idea: Make a RL model which can teach itself whether or not to mow the lawn, so think +3 for successfully mowing, 0 for not mowing, -5 partly mowing
rewardMow = 5
rewardNoMow = -4
rewardPartMow = -11

#States: based on two parameters, battery (high or low) and rain (high >=50% or low<50%)
states = [
    #[battery, rain]
    ("high", "high"),
    ("high", "low"),
    ("low", "high"),
    ("low", "low")
]

#Actions: include all possible actions from each
actions = {
    ("high", "high"): ["mow", "noMow"],
    ("high", "low"): ["mow", "noMow"],
    ("low", "high"): ["mow", "noMow"],
    ("low", "low"): ["mow", "noMow"]
}

#Transitions, map out possibilities(high to low for rain means it rained)
transitions = {
  (("high", "high"), "mow"): [(("high", "high"), 0.5, rewardMow), (("high", "low"), 0.2, rewardPartMow), (("low",  "high"), 0.2, rewardMow), (("low", "low"), 0.1, rewardPartMow)],
  (("high", "high"), "noMow"): [(("high", "high"), 0.5, rewardNoMow), (("high", "low"), 0.5, rewardNoMow)],

  (("high", "low"), "mow"): [(("high", "low"), 0.8, rewardMow), (("low",  "low"), 0.2, rewardMow)],
  (("high", "low"), "noMow"): [(("high", "low"), 0.7, rewardNoMow), (("high", "high"), 0.2, rewardNoMow)],

  (("low", "high"), "mow"): [(("low", "high"), 0.5, rewardMow), (("high", "low"), 0.05, rewardPartMow), (("low",  "low"), 0.1, rewardPartMow), (("high", "high"), 0.15, rewardPartMow)],
  (("low", "high"), "noMow"): [(("low", "high"), 0.5, rewardNoMow), (("low", "low"), 0.5, rewardNoMow)],

  (("low", "low"), "mow"): [(("high", "high"), 0.1, rewardPartMow), (("high", "low"), 0.2, rewardPartMow), (("low",  "high"), 0.2, rewardMow), (("low", "low"), 0.5, rewardMow)],
  (("low", "low"), "noMow"): [(("low", "low"), 0.9, rewardNoMow), (("low", "high"), 0.1, rewardNoMow)],
}

def trainer(states, actions, transitions, episodes, gamma = 0.99, theta = 0.001):
  V = {s: 0 for s in states}
  policy = {s: None for s in states}

  for i in range (0, episodes):
    delta = 0
    for s in states:
      actVals = {}
      for a in actions[s]:
        tot = 0
        for (s_next, prob, reward) in transitions[(s, a)]:
          tot += prob * (reward + gamma * V[s_next])
        actVals[a] = tot
      maxAct = max(actVals, key = actVals.get)
      delta = max(delta, abs(V[s] - actVals[maxAct]))
      V[s] = actVals[maxAct]
      policy[s] = maxAct
    if(delta<=theta):
      break

  return V,policy

In [ ]:
V,policy = trainer(states, actions, transitions, 100)
policy[("low","high")]

'mow'